In [1]:
import pandas as pd
import numpy as np
import math
import scipy
from scipy.stats import norm
from dataclasses import dataclass
pd.options.mode.chained_assignment = None

# Trade Quantities

* Sell $N_p = 1000$ put-spread units consisting of:
  * **Short put at $K_2$** (5% OTM put)
  * **Long put at $K_1$** (10% OTM put)

* Buy $N_c$ call contracts at strike $K_3$ (5% OTM call)

* Cash leftover: $C_0$

## Underlying

* SPY price:
  * At initiation: $S_0$
  * One week before expiry: $S_t$

## Option Values

### Put Spread

* At initiation:
  * Short put value: $V_{p2,0}$
  * Long put value: $V_{p1,0}$

* At time $t$:
  * $V_{p2,t}$
  * $V_{p1,t}$

### Call

* At initiation: $V_{c,0}$
* At time $t$: $V_{c,t}$

---

## Premium-Financing Constraint

Premium collected from selling the $K_2/K_1$ put spread finances the purchase of 5% OTM calls plus leftover cash:

$$
1000 \cdot 100 \cdot (V_{p2,0} - V_{p1,0}) = N_c \cdot 100 \cdot V_{c,0} + C_0
$$

Hence,

$$
N_c = \frac{1000 \cdot (V_{p2,0} - V_{p1,0}) - C_0/100}{V_{c,0}}
$$

---

## P&L One Week Before Expiry

Total mark-to-market P&L at time $t$ (A single options contract for SPY represents 100 shares of the SPDR S&P 500 ETF Trust):

$$
\text{PNL}_t = \underbrace{1000 \cdot 100 \cdot \left[(V_{p2,0} - V_{p2,t}) - (V_{p1,0} - V_{p1,t})\right]}_{\text{Put Spread P\&L}} + \underbrace{N_c \cdot 100 \cdot (V_{c,t} - V_{c,0})}_{\text{Long Call P\&L}} + \underbrace{C_0(e^{rt} - 1)}_{\text{Interest on Cash}}
$$

### Percentage Return Relative to Put-Spread Premium Collected

**Put Spread Premium:**

$$
\text{Put Spread Premium} = 1000 \cdot 100 \cdot (V_{p2,0} - V_{p1,0})
$$

**Percentage Return with respect to put premium collected:**

$$
\text{Percentage Return}(\%) = \frac{\text{PNL}_t \cdot 100}{{\text{Put Spread Premium at Time 0}}}
$$

In [2]:
class OptionsPricingModel:
    """Options pricing model for SPY options pricing"""
    
    def __init__(self, spot_price, risk_free_rate, dividend_yield):
        self.S0 = spot_price
        self.r = risk_free_rate
        self.q = dividend_yield  # SPY dividend yield
        
    def black_scholes_price(self, K, T, sigma, option_type='call'):
        """
        Calculate Black-Scholes price for European options
        
        Parameters:
        K: Strike price
        T: Time to maturity (in years)
        sigma: Implied volatility
        option_type: 'call' or 'put'
        """
        d1 = (np.log(self.S0 / K) + (self.r - self.q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        
        if option_type == 'call':
            price = self.S0 * np.exp(-self.q * T) * norm.cdf(d1) - K * np.exp(-self.r * T) * norm.cdf(d2)
        else:  # put
            price = K * np.exp(-self.r * T) * norm.cdf(-d2) - self.S0 * np.exp(-self.q * T) * norm.cdf(-d1)
        
        return price
    
    
    def crr_binomial_price(self, K, T, sigma, option_type='call', steps=1000, american=True):
        """
        Calculate option price using Cox-Ross-Rubinstein binomial tree
        
        Parameters:
        K: Strike price
        T: Time to maturity (in years)
        sigma: Volatility
        option_type: 'call' or 'put'
        steps: Number of time steps in the binomial tree
        american: True for American options, False for European
        """
        dt = T / steps  # Time increment
        
        # CRR parameters
        u = np.exp(sigma * np.sqrt(dt))  # Up factor
        d = 1 / u  # Down factor
        p = (np.exp((self.r - self.q) * dt) - d) / (u - d)  # Risk-neutral probability
        discount = np.exp(-self.r * dt)  # Discount factor per step
        
        # Initialize asset prices at maturity
        ST = np.zeros(steps + 1)
        for i in range(steps + 1):
            ST[i] = self.S0 * (u ** (steps - i)) * (d ** i)
        
        # Initialize option values at maturity
        if option_type == 'call':
            option_values = np.maximum(ST - K, 0)
        else:  # put
            option_values = np.maximum(K - ST, 0)
        
        # Backward induction through the tree
        for step in range(steps - 1, -1, -1):
            for i in range(step + 1):
                # Current stock price at this node
                S = self.S0 * (u ** (step - i)) * (d ** i)
                
                # Continuation value (discounted expected value)
                continuation_value = discount * (p * option_values[i] + (1 - p) * option_values[i + 1])
                
                # Early exercise value
                if option_type == 'call':
                    exercise_value = max(S - K, 0)
                else:  # put
                    exercise_value = max(K - S, 0)
                
                # For American options, take the maximum
                if american:
                    option_values[i] = max(continuation_value, exercise_value)
                else:
                    option_values[i] = continuation_value
        
        return option_values[0]

In [3]:
@dataclass
class TradeInputs:
    # Market / model inputs
    spot_price: float
    risk_free_rate: float
    dividend_yield: float
    time_to_maturity: float  # in years

    # Put Spread (SELL higher strike, BUY lower strike)
    put_spread_contracts: int

    put_short_strike: float      # e.g., 95% moneyness
    put_short_iv: float
    put_short_market_price: float

    put_long_strike: float       # e.g., 90% moneyness
    put_long_iv: float
    put_long_market_price: float

    # Call (BUY) 105% moneyness
    call_strike: float
    call_iv: float
    call_market_price: float

    # Binomial settings
    binomial_steps: int
    american: bool


class PutSpreadWithCallInitialSetUpAnalyzer:
    """
    Encapsulates the full pricing + report logic
    for the put spread (sell put, buy lower put) / long call trade.
    """

    def __init__(self, inputs: TradeInputs):
        self.inputs = inputs
        # Initialize the underlying pricing model
        self.model = OptionsPricingModel(
            spot_price=inputs.spot_price,
            risk_free_rate=inputs.risk_free_rate,
            dividend_yield=inputs.dividend_yield,
        )

    def compute(self) -> dict:
        """
        Run all pricing steps and return a dictionary of results
        (no printing here – just pure calculation).
        """
        i = self.inputs

        # === BS pricing ===
        put_short_price_bs = self.model.black_scholes_price(
            K=i.put_short_strike,
            T=i.time_to_maturity,
            sigma=i.put_short_iv,
            option_type='put'
        )

        put_long_price_bs = self.model.black_scholes_price(
            K=i.put_long_strike,
            T=i.time_to_maturity,
            sigma=i.put_long_iv,
            option_type='put'
        )

        call_price_bs = self.model.black_scholes_price(
            K=i.call_strike,
            T=i.time_to_maturity,
            sigma=i.call_iv,
            option_type='call'
        )

        # === Binomial pricing ===
        put_short_price_binom = self.model.crr_binomial_price(
            K=i.put_short_strike,
            T=i.time_to_maturity,
            sigma=i.put_short_iv,
            option_type='put',
            steps=i.binomial_steps,
            american=i.american
        )

        put_long_price_binom = self.model.crr_binomial_price(
            K=i.put_long_strike,
            T=i.time_to_maturity,
            sigma=i.put_long_iv,
            option_type='put',
            steps=i.binomial_steps,
            american=i.american
        )

        call_price_binom = self.model.crr_binomial_price(
            K=i.call_strike,
            T=i.time_to_maturity,
            sigma=i.call_iv,
            option_type='call',
            steps=i.binomial_steps,
            american=i.american
        )

        # === Step 5: Trade economics (using market prices for sizing) ===

        # Net premium from put spread (sell high strike, buy low strike)
        premium_collected_short_put = i.put_short_market_price * i.put_spread_contracts * 100
        premium_paid_long_put       = i.put_long_market_price  * i.put_spread_contracts * 100
        put_spread_premium          = premium_collected_short_put - premium_paid_long_put

        # Max number of call contracts that can be afforded with put spread premium
        call_contracts = max(0, int(put_spread_premium // (i.call_market_price * 100)))

        # Premium paid for calls using that size
        premium_paid = i.call_market_price * call_contracts * 100

        # Net premium 
        net_premium = put_spread_premium - premium_paid

        return {
            "spot_price": i.spot_price,
            "risk_free_rate": i.risk_free_rate,
            "dividend_yield": i.dividend_yield,
            "time_to_maturity": i.time_to_maturity,
            "binomial_steps": i.binomial_steps,

            # Short Put
            "put_short_strike": i.put_short_strike,
            "put_short_iv": i.put_short_iv,
            "put_short_contracts": i.put_spread_contracts,
            "put_short_market_price": i.put_short_market_price,
            "put_short_price_bs": put_short_price_bs,
            "put_short_price_binom": put_short_price_binom,
            "premium_collected_short_put": premium_collected_short_put,

            # Long Put
            "put_long_strike": i.put_long_strike,
            "put_long_iv": i.put_long_iv,
            "put_long_contracts": i.put_spread_contracts,
            "put_long_market_price": i.put_long_market_price,
            "put_long_price_bs": put_long_price_bs,
            "put_long_price_binom": put_long_price_binom,
            "premium_paid_long_put": premium_paid_long_put,

            # Call
            "call_strike": i.call_strike,
            "call_iv": i.call_iv,
            "call_market_price": i.call_market_price,
            "call_price_bs": call_price_bs,
            "call_price_binom": call_price_binom,

            # Trade economics based on the market price at T0
            "put_spread_premium": put_spread_premium,
            "call_contracts": call_contracts,
            "premium_paid": premium_paid,
            "net_premium": net_premium,
        }

    def print_report(self) -> None:
        """
        Pretty-print the same info the script printed,
        using the results from compute().
        """
        r = self.compute()

        print("=== TRADE SETUP ===")
        print(f"Spot Price: ${r['spot_price']:.2f}")
        print(f"Risk-free rate: {r['risk_free_rate']*100:.2f}%")
        print(f"Dividend yield: {r['dividend_yield']*100:.2f}%")
        print(f"Time to maturity: {r['time_to_maturity']:.4f} years")
        print(f"Binomial steps: {r['binomial_steps']}")
        print(f"Sizing based on: MARKET PRICES at T0")

        print("\n--- PUT SPREAD ---")
        print(f"\nSHORT PUT (Higher Strike - e.g., 95% Moneyness)")
        print(f"  Strike: ${r['put_short_strike']:.2f}")
        print(f"  Implied Vol: {r['put_short_iv']*100:.2f}%")
        print(f"  Contracts: {r['put_short_contracts']}")
        print(f"  Market price per option:      ${r['put_short_market_price']:.4f}")
        print(f"  BS price per option:          ${r['put_short_price_bs']:.4f}")
        print(f"  Binomial price per option:    ${r['put_short_price_binom']:.4f}")
        print(f"  Premium collected (market):   ${r['premium_collected_short_put']:,.2f}")

        print(f"\nLONG PUT (Lower Strike - e.g., 90% Moneyness)")
        print(f"  Strike: ${r['put_long_strike']:.2f}")
        print(f"  Implied Vol: {r['put_long_iv']*100:.2f}%")
        print(f"  Contracts: {r['put_long_contracts']}")
        print(f"  Market price per option:      ${r['put_long_market_price']:.4f}")
        print(f"  BS price per option:          ${r['put_long_price_bs']:.4f}")
        print(f"  Binomial price per option:    ${r['put_long_price_binom']:.4f}")
        print(f"  Premium paid (market):        ${r['premium_paid_long_put']:,.2f}")

        print(f"\nPUT SPREAD NET PREMIUM (market): ${r['put_spread_premium']:,.2f}")

        print("\n--- CALL (BUY) 105% Moneyness ---")
        print(f"Strike: ${r['call_strike']:.2f}")
        print(f"Implied Vol: {r['call_iv']*100:.2f}%")
        print(f"Contracts bought (market-sized): {r['call_contracts']}")
        print(f"  Market price per option:      ${r['call_market_price']:.4f}")
        print(f"  BS price per option:          ${r['call_price_bs']:.4f}")
        print(f"  Binomial price per option:    ${r['call_price_binom']:.4f}")
        print(f"  Premium paid (market):        ${r['premium_paid']:,.2f}")

        print("\n=== NET PREMIUM (based on market prices) ===")
        print(f"Put Spread Premium:    ${r['put_spread_premium']:,.2f}")
        print(f"Call Premium Paid:     ${r['premium_paid']:,.2f}")
        print(f"Net Premium:           ${r['net_premium']:,.2f}")

        print("\n=== MODEL CHECK ===")
        print(
            "Short Put:  BS vs Binomial =",
            round(r['put_short_price_bs'], 4), "vs", round(r['put_short_price_binom'], 4)
        )
        print(
            "Long Put:   BS vs Binomial =",
            round(r['put_long_price_bs'], 4), "vs", round(r['put_long_price_binom'], 4)
        )
        print(
            "Call:       BS vs Binomial =",
            round(r['call_price_bs'], 4), "vs", round(r['call_price_binom'], 4)
        )


In [4]:
def compute_trade_pnl(
    n_put_spread_contracts: int,
    n_call_contracts: int,
    V_p_short_0: float,
    V_p_short_t: float,
    V_p_long_0: float,
    V_p_long_t: float,
    V_c_0: float,
    V_c_t: float,
    net_premium: float,        # C0 (cash leftover)
    risk_free_rate: float,     # r
    rf_compounding_period: float    # t in years
) -> tuple:
    """
    Compute the PNL of the put spread + long call strategy.

    Formula used:
        PNL_t =
            + n_put_spread_contracts * 100 * [(V_p_short_0 - V_p_short_t) - (V_p_long_0 - V_p_long_t)]
            + n_call_contracts * 100 * (V_c_t - V_c_0)
            + net_premium * (e^(r*t) - 1)

    Where:
        n_put_spread_contracts : number of put spread units (each unit = 1 short put + 1 long put)
        n_call_contracts       : number of long call option contracts
        V_p_short_0, V_p_short_t : initial and final short put prices (higher strike, e.g., 95%)
        V_p_long_0, V_p_long_t   : initial and final long put prices (lower strike, e.g., 90%)
        V_c_0, V_c_t           : initial and final call prices
        net_premium            : net premium (cash leftover after financing calls)
        risk_free_rate         : r
        rf_compounding_period  : t in years

    Returns:
        pnl_total      : Total PnL of the strategy
        pnl_percentage : PnL as percentage of put spread premium collected
    """

    # --- PNL from put spread ---
    pnl_short_put = n_put_spread_contracts * 100 * (V_p_short_0 - V_p_short_t)
    
    # Long put P&L (we bought this, so profit when price increases)
    pnl_long_put = n_put_spread_contracts * 100 * (V_p_long_t - V_p_long_0)
    
    # Net put spread P&L
    term_put_spread = pnl_short_put + pnl_long_put

    # --- PNL from long calls ---
    term_call = n_call_contracts * 100 * (V_c_t - V_c_0)

    # --- Carry on cash premium ---
    term_carry = net_premium * (math.exp(risk_free_rate * rf_compounding_period) - 1)

    # --- Total PnL ---
    pnl_total = term_put_spread + term_call + term_carry

    # --- Percentage return relative to put spread premium collected ---
    put_spread_premium_collected = n_put_spread_contracts * 100 * (V_p_short_0 - V_p_long_0)
    pnl_percentage = (pnl_total / put_spread_premium_collected) * 100 if put_spread_premium_collected != 0 else 0

    return pnl_total, pnl_percentage


def put_spread_call_greeks(
    n_put_spread,
    n_call,
    put_short_delta,
    put_short_gamma,
    put_short_vega,
    put_short_theta,
    put_short_rho,
    put_long_delta,
    put_long_gamma,
    put_long_vega,
    put_long_theta,
    put_long_rho,
    call_delta,
    call_gamma,
    call_vega,
    call_theta,
    call_rho,
    contract_size=100,
):
    """
    Compute portfolio Greeks for a put spread + long call strategy.

    Parameters:
        n_put_spread : number of put spread units (each = 1 short put + 1 long put)
                      - Use positive value since we define the spread as a unit
        n_call      : number of long call contracts (positive for long position)
        
        put_short_* : Greeks for the short put (higher strike, e.g., 95%)
        put_long_*  : Greeks for the long put (lower strike, e.g., 90%)
        call_*      : Greeks for the long call
        
        contract_size : usually 100 for US equity options
    """

    # Put spread Greeks = (short put contribution) + (long put contribution)
    # Short put: negative contribution (we're short)
    # Long put: positive contribution (we're long)
    
    delta = contract_size * (
        -n_put_spread * put_short_delta +  # Short put (negative)
        n_put_spread * put_long_delta +    # Long put (positive)
        n_call * call_delta                # Long call (positive)
    )
    
    gamma = contract_size * (
        -n_put_spread * put_short_gamma +
        n_put_spread * put_long_gamma +
        n_call * call_gamma
    )
    
    vega = contract_size * (
        -n_put_spread * put_short_vega +
        n_put_spread * put_long_vega +
        n_call * call_vega
    )
    
    theta = contract_size * (
        -n_put_spread * put_short_theta +
        n_put_spread * put_long_theta +
        n_call * call_theta
    )
    
    rho = contract_size * (
        -n_put_spread * put_short_rho +
        n_put_spread * put_long_rho +
        n_call * call_rho
    )

    return {
        "delta": delta,
        "gamma": gamma,
        "vega": vega,
        "theta": theta,
        "rho": rho,
    }

In [5]:
# Load 5% & 10% OTM option data for the given date and define the rf and dividend rate
five_percent_otm_options_path = '../data/Option/eod_option_5%_10%_OTM_2025-04-09.csv'
df = pd.read_csv(five_percent_otm_options_path)
risk_free_rate=0.0433
dividend_yield=0.0127
time_to_maturity = 30/365

# Separate calls and puts
call = df[df["call_put"] == "C"].iloc[0]     # 5% OTM call
puts = df[df["call_put"] == "P"]            # two puts

put_short = puts.iloc[0]   # e.g. 5% OTM put
put_long  = puts.iloc[1]   # e.g. 10% OTM put

# Spot price (same for all rows)
spot_price = df["underlying_price"].iloc[0]

# ---------------- CALL (5% OTM) ----------------
call_market_price = call["price"]
call_iv     = call["iv"]
call_strike = call["price_strike"]
call_delta  = call["delta"]
call_gamma  = call["gamma"]
call_vega   = call["vega"]
call_theta  = call["theta"]
call_rho    = call["rho"]

# ---------------- PUT SHORT (5% OTM) ----------------
put_short_market_price = put_short["price"]
put_short_iv      = put_short["iv"]
put_short_strike  = put_short["price_strike"]
put_short_delta   = put_short["delta"]
put_short_gamma   = put_short["gamma"]
put_short_vega    = put_short["vega"]
put_short_theta   = put_short["theta"]
put_short_rho     = put_short["rho"]

# ---------------- PUT LONG (10% OTM) ----------------
put_long_market_price = put_long["price"]
put_long_iv       = put_long["iv"]
put_long_strike   = put_long["price_strike"]
put_long_delta    = put_long["delta"]
put_long_gamma    = put_long["gamma"]
put_long_vega     = put_long["vega"]
put_long_theta    = put_long["theta"]
put_long_rho      = put_long["rho"]

# ---------------- PRINT RESULTS ----------------
print(f"Spot Price: {spot_price}")

print("\n--- CALL (5% OTM) ---")
print(f"Call Market Price: {call_market_price}")
print(f"IV:     {call_iv}")
print(f"Strike: {call_strike}")
print(f"Delta:  {call_delta}")
print(f"Gamma:  {call_gamma}")
print(f"Vega:   {call_vega}")
print(f"Theta:  {call_theta}")
print(f"Rho:    {call_rho}")

print("\n--- PUT SHORT (5% OTM) ---")
print(f"Put Short Market Price: {put_short_market_price}")
print(f"IV:     {put_short_iv}")
print(f"Strike: {put_short_strike}")
print(f"Delta:  {put_short_delta}")
print(f"Gamma:  {put_short_gamma}")
print(f"Vega:   {put_short_vega}")
print(f"Theta:  {put_short_theta}")
print(f"Rho:    {put_short_rho}")

print("\n--- PUT LONG (10% OTM) ---")
print(f"Put Long Market Price: {put_long_market_price}")
print(f"IV:     {put_long_iv}")
print(f"Strike: {put_long_strike}")
print(f"Delta:  {put_long_delta}")
print(f"Gamma:  {put_long_gamma}")
print(f"Vega:   {put_long_vega}")
print(f"Theta:  {put_long_theta}")
print(f"Rho:    {put_long_rho}")



Spot Price: 543.37

--- CALL (5% OTM) ---
Call Market Price: 4.225
IV:     0.210945
Strike: 571
Delta:  0.228211
Gamma:  0.009197
Vega:   0.470783
Theta:  -0.176452
Rho:    0.098448

--- PUT SHORT (5% OTM) ---
Put Short Market Price: 8.475
IV:     0.323284
Strike: 516
Delta:  -0.264147
Gamma:  0.006544
Vega:   0.51151
Theta:  -0.262369
Rho:    -0.111529

--- PUT LONG (10% OTM) ---
Put Long Market Price: 4.425
IV:     0.39027
Strike: 485
Delta:  -0.13644
Gamma:  0.003617
Vega:   0.343551
Theta:  -0.215429
Rho:    -0.059738


In [6]:
inputs = TradeInputs(
    spot_price=spot_price,
    risk_free_rate=risk_free_rate,
    dividend_yield=dividend_yield,
    time_to_maturity=time_to_maturity,

    put_spread_contracts = 1000,

    put_short_strike=put_short_strike,
    put_short_iv=put_short_iv,
    put_short_market_price=put_short_market_price,
    

    put_long_strike=put_long_strike,
    put_long_iv=put_long_iv,
    put_long_market_price=put_long_market_price,

    call_strike=call_strike,
    call_iv=call_iv,
    call_market_price=call_market_price,

    binomial_steps=1000,
    american=True,
)

analyzer = PutSpreadWithCallInitialSetUpAnalyzer(inputs=inputs)
analyzer.print_report()
initial_set_up = analyzer.compute()

initial_set_up

=== TRADE SETUP ===
Spot Price: $543.37
Risk-free rate: 4.33%
Dividend yield: 1.27%
Time to maturity: 0.0822 years
Binomial steps: 1000
Sizing based on: MARKET PRICES at T0

--- PUT SPREAD ---

SHORT PUT (Higher Strike - e.g., 95% Moneyness)
  Strike: $516.00
  Implied Vol: 32.33%
  Contracts: 1000
  Market price per option:      $8.4750
  BS price per option:          $8.4602
  Binomial price per option:    $8.4911
  Premium collected (market):   $847,500.00

LONG PUT (Lower Strike - e.g., 90% Moneyness)
  Strike: $485.00
  Implied Vol: 39.03%
  Contracts: 1000
  Market price per option:      $4.4250
  BS price per option:          $4.4330
  Binomial price per option:    $4.4408
  Premium paid (market):        $442,500.00

PUT SPREAD NET PREMIUM (market): $405,000.00

--- CALL (BUY) 105% Moneyness ---
Strike: $571.00
Implied Vol: 21.09%
Contracts bought (market-sized): 958
  Market price per option:      $4.2250
  BS price per option:          $4.1938
  Binomial price per option:    $

{'spot_price': np.float64(543.37),
 'risk_free_rate': 0.0433,
 'dividend_yield': 0.0127,
 'time_to_maturity': 0.0821917808219178,
 'binomial_steps': 1000,
 'put_short_strike': np.int64(516),
 'put_short_iv': np.float64(0.323284),
 'put_short_contracts': 1000,
 'put_short_market_price': np.float64(8.475),
 'put_short_price_bs': np.float64(8.460236074876917),
 'put_short_price_binom': np.float64(8.491084290464878),
 'premium_collected_short_put': np.float64(847500.0),
 'put_long_strike': np.int64(485),
 'put_long_iv': np.float64(0.39027),
 'put_long_contracts': 1000,
 'put_long_market_price': np.float64(4.425),
 'put_long_price_bs': np.float64(4.433012136525775),
 'put_long_price_binom': np.float64(4.440784051061745),
 'premium_paid_long_put': np.float64(442500.0),
 'call_strike': np.int64(571),
 'call_iv': np.float64(0.210945),
 'call_market_price': np.float64(4.225),
 'call_price_bs': np.float64(4.193847130448077),
 'call_price_binom': np.float64(4.191387904323273),
 'put_spread_premiu

In [7]:
put_spread_call_greeks(
    initial_set_up['put_short_contracts'],
    initial_set_up['call_contracts'],
    put_short_delta,
    put_short_gamma,
    put_short_vega,
    put_short_theta,
    put_short_rho,
    put_long_delta,
    put_long_gamma,
    put_long_vega,
    put_long_theta,
    put_long_rho,
    call_delta,
    call_gamma,
    call_vega,
    call_theta,
    call_rho,
    contract_size=100,
)

{'delta': np.float64(34633.31380000001),
 'gamma': np.float64(588.3726),
 'vega': np.float64(28305.111399999994),
 'theta': np.float64(-12210.101599999996),
 'rho': np.float64(14610.418399999997)}

# **Scenario Analysis for 1-month Put Spread (90%-95%) & 105% Call Long**

### **Trade Structure (as of 4/9/2025 afternoon)**

* **Buy 1000× 1-month 90% moneyness SPY puts**

  * Initial IV (10% OTM put): **0.39027**

* **Sell 1000× 1-month 95% moneyness SPY puts**

  * Initial IV (5% OTM put): **0.323**
* **Buy 1000× 1-month 105% moneyness SPY calls (using the premium)**

  * Initial IV (5% OTM call): **0.211**

### **Objective**

Project **1-week-before-expiry P&L**, under:

1. **Spot moves (over 3 weeks):**
   +15%, +10%, +5%, +2.5%, 0%, −2.5%, −5%, −10% -15%
2. **Implied volatility adjustments:**
   Need multiple reasonable IV paths for both the **OTM call** and **OTM put**.

---

# **Scenario Grid (IV % Changes)**

All IV changes are relative to the initial IVs:

* **Call IV₀ = 21.1%**
* **Put Short IV₀ = 32.3%**
* **Put Long IV₀ = 39.027%**

Values are expressed as **percentage change of implied volatility**, not absolute points.

| Spot Change | Scenario       | ΔIV (105% Call) | ΔIV (95% Put) | ΔIV (90% Put) | Short Market Reasoning                                                                                      |
| ----------: | -------------- | --------------- | ------------- | ------------- | ----------------------------------------------------------------------------------------------------------- |
|    **+15%** | vol_collapse   | −9%             | −25%          | −32%          | Violent melt-up; crash premium dumped, skew sharply flattens as downside hedges are unwound.                |
|             | normal_decline | −6%             | −17%          | −22%          | Strong but orderly rally; systematic vol sellers and lack of demand for puts drive broad vol roll-down.     |
|             | mixed_regime   | −3%             | −10%          | −13%          | Rally continues but some macro uncertainty remains; skew compresses, but downside still holds residual bid. |
|             | persistent_vol | 0%              | −4%           | −6%           | Price rips higher but realized vol stays elevated; calls hold bid while downside IV slowly normalizes.      |
|    **+10%** | vol_collapse   | −7%             | −18%          | −26%          | Continued risk-on; short-gamma covering and structured vol supply crush downside tails.                     |
|             | normal_decline | −5%             | −13%          | −18%          | Healthy trend up; hedging demand fades and both wings drift lower, more so in puts.                         |
|             | mixed_regime   | −2%             | −7%           | −10%          | Grind higher with pockets of macro risk; skew eases but investors keep some tail protection.                |
|             | persistent_vol | +2%             | −2%           | −4%           | Rally but with elevated realized swings; upside optionality bid while downside slowly cheapens.             |
|     **+5%** | vol_collapse   | −5%             | −12%          | −18%          | Relief rally after prior stress; vol sellers lean in and downside IV compresses meaningfully.               |
|             | normal_decline | −3%             | −7%           | −10%          | Moderate grind up; steady reduction in put demand and skew.                                                 |
|             | mixed_regime   | 0%              | 0%            | −2%           | Quiet drift higher; term-structure roll-down offset by mild macro worries, leaving surface unchanged.       |
|             | persistent_vol | +3%             | +3%           | +5%           | Rally occurs in choppy tape; both wings retain a premium due to higher realized vol-of-vol.                 |
|   **+2.5%** | vol_collapse   | −2%             | −5%           | −7%           | Small extension of risk-on move; dealers supply vol and downside IV softens.                                |
|             | normal_decline | −1%             | −2%           | −3%           | Mild upside; theta decay plus roll-down gradually cheapen both call and put IVs.                            |
|             | mixed_regime   | +2%             | +4%           | +6%           | Spot drifts up but data risk looms; investors pay for short-dated gamma on both sides.                      |
|             | persistent_vol | +5%             | +7%           | +10%          | Choppy, headline-driven rise; volatility of volatility stays high and both wings remain well bid.           |
|      **0%** | vol_collapse   | −2%             | −2%           | −3%           | Market trades sideways with low realized vol; surface mean-reverts slightly lower.                          |
|             | normal_decline | 0%              | 0%            | 0%            | Balanced flows; theta and roll-down largely priced in, surface effectively flat.                            |
|             | mixed_regime   | +3%             | +5%           | +7%           | Range-bound but event risk ahead; gamma and skew see a modest bid.                                          |
|             | persistent_vol | +6%             | +10%          | +13%          | Sticky uncertainty; elevated hedging and gamma-trading keep both wings rich, especially downside.           |
|   **−2.5%** | vol_collapse   | 0%              | +2%           | +4%           | Small pullback after strong run; modest put demand, but no real fear.                                       |
|             | normal_decline | +1%             | +4%           | +7%           | Dip is bought but hedgers add some protection; skew steepens a bit.                                         |
|             | mixed_regime   | +3%             | +9%           | +13%          | Repeated minor selloffs; investors start to chase downside protection and skew.                             |
|             | persistent_vol | +5%             | +13%          | +18%          | Choppy down-tape; systematic hedging flows lift both wings with a stronger bid for puts.                    |
|     **−5%** | vol_collapse   | +2%             | +5%           | +9%           | Initial flush, then stabilization; downside IV marks higher but not aggressively as dip buyers step in.     |
|             | normal_decline | +3%             | +9%           | +13%          | Classic risk-off day; hedging demand increases and skew moves steeper.                                      |
|             | mixed_regime   | +5%             | +14%          | +20%          | Repeated tests of lower levels; dealers and real money both pay up for crash protection.                    |
|             | persistent_vol | +8%             | +20%          | +28%          | Sustained volatile drawdown; vol-of-vol high, downside IV re-prices toward stress levels.                   |
|    **−10%** | vol_collapse   | +2%             | +6%           | +10%          | Large move but largely anticipated (e.g., event); vol rises, yet less than historical panic norms.          |
|             | normal_decline | +5%             | +16%          | +22%          | Strong hedging flows after a sharp selloff; puts command sizeable crash premium.                            |
|             | mixed_regime   | +8%             | +24%          | +32%          | Fear re-emerges; vol buyers dominate and skew steepens materially.                                          |
|             | persistent_vol | +12%            | +30%          | +40%          | Extended risk-off environment; downside vol moves into clear stress-regime territory.                       |
|    **−15%** | vol_collapse   | +3%             | +8%           | +13%          | Deep correction but with earlier hedges in place; vol rises but remains below full-blown panic.             |
|             | normal_decline | +7%             | +22%          | +30%          | Crisis-style hedging; crash-risk premium jumps and skew becomes very steep.                                 |
|             | mixed_regime   | +11%            | +32%          | +42%          | High-stress environment; repeated gaps and liquidity concerns drive heavy demand for puts.                  |
|             | persistent_vol | +16%            | +40%          | +50%          | Near-panic regime (VIX mid-40s); forced selling and crash hedging send downside IV to crisis levels.        |


In [8]:
iv_scenarios = {
    0.15: {
        "vol_collapse":   {"call_iv_change": -0.09, "put_short_iv_change": -0.25, "put_long_iv_change": -0.32},
        "normal_decline": {"call_iv_change": -0.06, "put_short_iv_change": -0.17, "put_long_iv_change": -0.22},
        "mixed_regime":   {"call_iv_change": -0.03, "put_short_iv_change": -0.10, "put_long_iv_change": -0.13},
        "persistent_vol": {"call_iv_change":  0.00, "put_short_iv_change": -0.04, "put_long_iv_change": -0.06},
    },
    0.10: {
        "vol_collapse":   {"call_iv_change": -0.07, "put_short_iv_change": -0.18, "put_long_iv_change": -0.26},
        "normal_decline": {"call_iv_change": -0.05, "put_short_iv_change": -0.13, "put_long_iv_change": -0.18},
        "mixed_regime":   {"call_iv_change": -0.02, "put_short_iv_change": -0.07, "put_long_iv_change": -0.10},
        "persistent_vol": {"call_iv_change":  0.02, "put_short_iv_change": -0.02, "put_long_iv_change": -0.04},
    },
    0.05: {
        "vol_collapse":   {"call_iv_change": -0.05, "put_short_iv_change": -0.12, "put_long_iv_change": -0.18},
        "normal_decline": {"call_iv_change": -0.03, "put_short_iv_change": -0.07, "put_long_iv_change": -0.10},
        "mixed_regime":   {"call_iv_change":  0.00, "put_short_iv_change":  0.00, "put_long_iv_change": -0.02},
        "persistent_vol": {"call_iv_change":  0.03, "put_short_iv_change":  0.03, "put_long_iv_change":  0.05},
    },
    0.025: {
        "vol_collapse":   {"call_iv_change": -0.02, "put_short_iv_change": -0.05, "put_long_iv_change": -0.07},
        "normal_decline": {"call_iv_change": -0.01, "put_short_iv_change": -0.02, "put_long_iv_change": -0.03},
        "mixed_regime":   {"call_iv_change":  0.02, "put_short_iv_change":  0.04, "put_long_iv_change":  0.06},
        "persistent_vol": {"call_iv_change":  0.05, "put_short_iv_change":  0.07, "put_long_iv_change":  0.10},
    },
    0.0: {
        "vol_collapse":   {"call_iv_change": -0.02, "put_short_iv_change": -0.02, "put_long_iv_change": -0.03},
        "normal_decline": {"call_iv_change":  0.00, "put_short_iv_change":  0.00, "put_long_iv_change":  0.00},
        "mixed_regime":   {"call_iv_change":  0.03, "put_short_iv_change":  0.05, "put_long_iv_change":  0.07},
        "persistent_vol": {"call_iv_change":  0.06, "put_short_iv_change":  0.10, "put_long_iv_change":  0.13},
    },
    -0.025: {
        "vol_collapse":   {"call_iv_change":  0.00, "put_short_iv_change":  0.02, "put_long_iv_change":  0.04},
        "normal_decline": {"call_iv_change":  0.01, "put_short_iv_change":  0.04, "put_long_iv_change":  0.07},
        "mixed_regime":   {"call_iv_change":  0.03, "put_short_iv_change":  0.09, "put_long_iv_change":  0.13},
        "persistent_vol": {"call_iv_change":  0.05, "put_short_iv_change":  0.13, "put_long_iv_change":  0.18},
    },
    -0.05: {
        "vol_collapse":   {"call_iv_change":  0.02, "put_short_iv_change":  0.05, "put_long_iv_change":  0.09},
        "normal_decline": {"call_iv_change":  0.03, "put_short_iv_change":  0.09, "put_long_iv_change":  0.13},
        "mixed_regime":   {"call_iv_change":  0.05, "put_short_iv_change":  0.14, "put_long_iv_change":  0.20},
        "persistent_vol": {"call_iv_change":  0.08, "put_short_iv_change":  0.20, "put_long_iv_change":  0.28},
    },
    -0.10: {
        "vol_collapse":   {"call_iv_change":  0.02, "put_short_iv_change":  0.06, "put_long_iv_change":  0.10},
        "normal_decline": {"call_iv_change":  0.05, "put_short_iv_change":  0.16, "put_long_iv_change":  0.22},
        "mixed_regime":   {"call_iv_change":  0.08, "put_short_iv_change":  0.24, "put_long_iv_change":  0.32},
        "persistent_vol": {"call_iv_change":  0.12, "put_short_iv_change":  0.30, "put_long_iv_change":  0.40},
    },
    -0.15: {
        "vol_collapse":   {"call_iv_change":  0.03, "put_short_iv_change":  0.08, "put_long_iv_change":  0.13},
        "normal_decline": {"call_iv_change":  0.07, "put_short_iv_change":  0.22, "put_long_iv_change":  0.30},
        "mixed_regime":   {"call_iv_change":  0.11, "put_short_iv_change":  0.32, "put_long_iv_change":  0.42},
        "persistent_vol": {"call_iv_change":  0.16, "put_short_iv_change":  0.40, "put_long_iv_change":  0.50},
    },
}



In [9]:
inputs = TradeInputs(
    spot_price=spot_price,
    risk_free_rate=risk_free_rate,
    dividend_yield=dividend_yield,
    time_to_maturity=time_to_maturity,

    put_spread_contracts = 1000,

    put_short_strike=put_short_strike,
    put_short_iv=put_short_iv,
    put_short_market_price=put_short_market_price,
    

    put_long_strike=put_long_strike,
    put_long_iv=put_long_iv,
    put_long_market_price=put_long_market_price,

    call_strike=call_strike,
    call_iv=call_iv,
    call_market_price=call_market_price,

    binomial_steps=1000,
    american=True,
)

analyzer = PutSpreadWithCallInitialSetUpAnalyzer(inputs=inputs)
analyzer.print_report()
initial_set_up = analyzer.compute()

initial_set_up

=== TRADE SETUP ===
Spot Price: $543.37
Risk-free rate: 4.33%
Dividend yield: 1.27%
Time to maturity: 0.0822 years
Binomial steps: 1000
Sizing based on: MARKET PRICES at T0

--- PUT SPREAD ---

SHORT PUT (Higher Strike - e.g., 95% Moneyness)
  Strike: $516.00
  Implied Vol: 32.33%
  Contracts: 1000
  Market price per option:      $8.4750
  BS price per option:          $8.4602
  Binomial price per option:    $8.4911
  Premium collected (market):   $847,500.00

LONG PUT (Lower Strike - e.g., 90% Moneyness)
  Strike: $485.00
  Implied Vol: 39.03%
  Contracts: 1000
  Market price per option:      $4.4250
  BS price per option:          $4.4330
  Binomial price per option:    $4.4408
  Premium paid (market):        $442,500.00

PUT SPREAD NET PREMIUM (market): $405,000.00

--- CALL (BUY) 105% Moneyness ---
Strike: $571.00
Implied Vol: 21.09%
Contracts bought (market-sized): 958
  Market price per option:      $4.2250
  BS price per option:          $4.1938
  Binomial price per option:    $

{'spot_price': np.float64(543.37),
 'risk_free_rate': 0.0433,
 'dividend_yield': 0.0127,
 'time_to_maturity': 0.0821917808219178,
 'binomial_steps': 1000,
 'put_short_strike': np.int64(516),
 'put_short_iv': np.float64(0.323284),
 'put_short_contracts': 1000,
 'put_short_market_price': np.float64(8.475),
 'put_short_price_bs': np.float64(8.460236074876917),
 'put_short_price_binom': np.float64(8.491084290464878),
 'premium_collected_short_put': np.float64(847500.0),
 'put_long_strike': np.int64(485),
 'put_long_iv': np.float64(0.39027),
 'put_long_contracts': 1000,
 'put_long_market_price': np.float64(4.425),
 'put_long_price_bs': np.float64(4.433012136525775),
 'put_long_price_binom': np.float64(4.440784051061745),
 'premium_paid_long_put': np.float64(442500.0),
 'call_strike': np.int64(571),
 'call_iv': np.float64(0.210945),
 'call_market_price': np.float64(4.225),
 'call_price_bs': np.float64(4.193847130448077),
 'call_price_binom': np.float64(4.191387904323273),
 'put_spread_premiu

In [10]:
def scenario_simulation(
    initial_set_up: dict,
    iv_scenarios: dict,
    model: type[OptionsPricingModel],
    pricing_model_used: str,
    time_left_to_expiry: float,
    rf_compounding_period: float
) -> dict:
    """
    Run spot + IV scenarios and return, for each (spot_move, scenario_name):
        - V_p_short_t: scenario short put price
        - V_p_long_t: scenario long put price
        - V_c_t: scenario call price
        - pnl_dollars: PnL of the strategy in dollars
        - pnl_pct: PnL as a percentage of |net_premium|
    """

    # -------------------------------------------- Unpack initial setup at T0 ----------------------------------------------
    initial_spot_price    = initial_set_up["spot_price"]
    n_put_spread_contracts = initial_set_up["put_long_contracts"]
    n_call_contracts      = initial_set_up["call_contracts"]

    put_long_strike       = initial_set_up["put_long_strike"]
    put_short_strike      = initial_set_up["put_short_strike"]
    call_strike           = initial_set_up["call_strike"]

    put_long_iv           = initial_set_up["put_long_iv"]
    put_short_iv          = initial_set_up["put_short_iv"]
    call_iv               = initial_set_up["call_iv"]

    net_premium           = initial_set_up["net_premium"]
    risk_free_rate        = initial_set_up["risk_free_rate"]
    dividend_yield        = initial_set_up["dividend_yield"]
    binomial_steps        = initial_set_up["binomial_steps"]

    # Initial prices at T0 (market)
    V_p_short_0 = initial_set_up["put_short_market_price"]
    V_p_long_0  = initial_set_up["put_long_market_price"]
    V_c_0       = initial_set_up["call_market_price"]

    # Normalize pricing model name for robustness
    pricing_model_used = pricing_model_used.lower()

    # Optional: get American flag if you want consistency with setup
    american = initial_set_up.get("american", True)

    # -------------------------------------------- Calculate PnL for each scenario ----------------------------------------------

    scenario_simulation_result: dict = {}

    # --- Loop over spot and IV scenarios ---
    for spot_move, scenario_dict in iv_scenarios.items():
        move_key = spot_move * 100  # could cast to int if you prefer

        # Ensure nested dict exists
        scenario_simulation_result[move_key] = {}

        # New spot is a percentage move from initial
        new_spot = initial_spot_price * (1 + spot_move)

        for scenario_name, iv_changes in scenario_dict.items():
            call_change       = iv_changes["call_iv_change"]
            put_short_change  = iv_changes["put_short_iv_change"]
            put_long_change   = iv_changes["put_long_iv_change"]

            # Apply IV changes
            new_put_short_iv  = put_short_iv * (1 + put_short_change)
            new_put_long_iv   = put_long_iv  * (1 + put_long_change)
            new_call_iv       = call_iv      * (1 + call_change)

            # Debug prints (optional)
            print("\n--- SPOT MOVE ---")
            print(f"Before spot: {initial_spot_price}")
            print(f"Change : {spot_move * 100}%")
            print(f"After  spot: {new_spot}")

            print("\n--- IMPLIED VOLATILITY CHANGES (SHORT PUT) ---")
            print(f"  Before: {put_short_iv}")
            print(f"  Change: {put_short_change * 100:+}%")
            print(f"  After : {new_put_short_iv}")

            print("\n--- IMPLIED VOLATILITY CHANGES (LONG PUT) ---")
            print(f"  Before: {put_long_iv}")
            print(f"  Change: {put_long_change * 100:+}%")
            print(f"  After : {new_put_long_iv}")

            print("\n----- CALL IV CHANGES -----")
            print(f"  Before: {call_iv}")
            print(f"  Change: {call_change * 100:+}%")
            print(f"  After : {new_call_iv}")

            # Build a pricing model at the new spot
            pricing_model = model(
                spot_price=new_spot,
                risk_free_rate=risk_free_rate,
                dividend_yield=dividend_yield,
            )

            # --------------------------------------- Price options under chosen model -------------------------------
            if pricing_model_used == "bs":
                V_p_short_t = pricing_model.black_scholes_price(
                    K=put_short_strike,
                    T=time_left_to_expiry,
                    sigma=new_put_short_iv,
                    option_type="put",
                )

                V_p_long_t = pricing_model.black_scholes_price(
                    K=put_long_strike,
                    T=time_left_to_expiry,
                    sigma=new_put_long_iv,
                    option_type="put",
                )

                V_c_t = pricing_model.black_scholes_price(
                    K=call_strike,
                    T=time_left_to_expiry,
                    sigma=new_call_iv,
                    option_type="call",
                )

            elif pricing_model_used == "binomial":
                V_p_short_t = pricing_model.crr_binomial_price(
                    K=put_short_strike,
                    T=time_left_to_expiry,
                    sigma=new_put_short_iv,
                    option_type="put",
                    steps=binomial_steps,
                    american=american,
                )

                V_p_long_t = pricing_model.crr_binomial_price(
                    K=put_long_strike,
                    T=time_left_to_expiry,
                    sigma=new_put_long_iv,
                    option_type="put",
                    steps=binomial_steps,
                    american=american,
                )

                V_c_t = pricing_model.crr_binomial_price(
                    K=call_strike,
                    T=time_left_to_expiry,
                    sigma=new_call_iv,
                    option_type="call",
                    steps=binomial_steps,
                    american=american,
                )

            else:
                raise ValueError(f"Unknown pricing_model_used: {pricing_model_used}")

            # Compute PnL of the strategy
            pnl, pnl_percentage = compute_trade_pnl(
                n_put_spread_contracts=n_put_spread_contracts,
                n_call_contracts=n_call_contracts,
                V_p_short_0=V_p_short_0,
                V_p_short_t=V_p_short_t,
                V_p_long_0=V_p_long_0,
                V_p_long_t=V_p_long_t,
                V_c_0=V_c_0,
                V_c_t=V_c_t,
                net_premium=net_premium,              # C0 (cash leftover)
                risk_free_rate=risk_free_rate,        # r
                rf_compounding_period=rf_compounding_period  # t in years
            )

            scenario_simulation_result[move_key][scenario_name] = {
                "S_0": initial_spot_price,
                "S_t": round(new_spot, 2),

                "put_long_iv_0":  round(put_long_iv, 4),
                "Change_in_put_long_iv (%)": round((new_put_long_iv / put_long_iv - 1) * 100, 4) if put_long_iv != 0 else None,
                "put_long_iv_t":  round(new_put_long_iv, 4),

                "put_short_iv_0": round(put_short_iv, 4),
                "Change_in_put_short_iv (%)": round((new_put_short_iv / put_short_iv - 1) * 100, 4) if put_short_iv != 0 else None,
                "put_short_iv_t": round(new_put_short_iv, 4),

                "call_iv_0": round(call_iv, 4),
                "Change_in_call_iv (%)": round((new_call_iv / call_iv - 1) * 100, 4) if call_iv != 0 else None,
                "call_iv_t": round(new_call_iv, 4),

                "V_p_long_0": round(V_p_long_0, 4),
                "Change_in_put_long_price (%)": round((V_p_long_t / V_p_long_0 - 1) * 100, 4) if V_p_long_0 != 0 else None,
                "V_p_long_t": round(V_p_long_t, 4),

                "V_p_short_0": round(V_p_short_0, 4),
                "Change_in_put_short_price (%)": round((V_p_short_t / V_p_short_0 - 1) * 100, 4) if V_p_short_0 != 0 else None,
                "V_p_short_t": round(V_p_short_t, 4),

                "V_c_0": round(V_c_0, 4),
                "Change_in_call_price (%)": round((V_c_t / V_c_0 - 1) * 100, 4) if V_c_0 != 0 else None,
                "V_c_t": round(V_c_t, 4),

                "pnl_dollars": round(pnl, 2),
                "pnl_percentage": round(pnl_percentage, 2),
            }

    return scenario_simulation_result


def simulation_result_to_df(scenario_simulation_result):
    """
    Convert a nested dictionary of the form:
        { move: { scenario: { field: value, ... } } }
    into a MultiIndex pandas DataFrame.
    
    Index levels:
        level 0 → move
        level 1 → scenario
    """
    
    # Flatten nested structure into tuple keys
    flat_dict = {
        (move, scenario): metrics
        for move, scenarios in scenario_simulation_result.items()
        for scenario, metrics in scenarios.items()
    }

    # Build DataFrame
    df = pd.DataFrame.from_dict(flat_dict, orient="index")

    # Set MultiIndex with names
    df.index = pd.MultiIndex.from_tuples(df.index, names=["Spot Move(%)", "scenario"])

    return df

In [11]:
scenario_simulation_result = scenario_simulation(
    initial_set_up,
    iv_scenarios,
    OptionsPricingModel,
    pricing_model_used ='binomial',
    time_left_to_expiry=7/365,
    rf_compounding_period=23/365
)

scenario_simulation_result 


--- SPOT MOVE ---
Before spot: 543.37
Change : 15.0%
After  spot: 624.8755

--- IMPLIED VOLATILITY CHANGES (SHORT PUT) ---
  Before: 0.323284
  Change: -25.0%
  After : 0.242463

--- IMPLIED VOLATILITY CHANGES (LONG PUT) ---
  Before: 0.39027
  Change: -32.0%
  After : 0.2653836

----- CALL IV CHANGES -----
  Before: 0.210945
  Change: -9.0%
  After : 0.19195995

--- SPOT MOVE ---
Before spot: 543.37
Change : 15.0%
After  spot: 624.8755

--- IMPLIED VOLATILITY CHANGES (SHORT PUT) ---
  Before: 0.323284
  Change: -17.0%
  After : 0.26832572

--- IMPLIED VOLATILITY CHANGES (LONG PUT) ---
  Before: 0.39027
  Change: -22.0%
  After : 0.30441060000000003

----- CALL IV CHANGES -----
  Before: 0.210945
  Change: -6.0%
  After : 0.19828829999999997

--- SPOT MOVE ---
Before spot: 543.37
Change : 15.0%
After  spot: 624.8755

--- IMPLIED VOLATILITY CHANGES (SHORT PUT) ---
  Before: 0.323284
  Change: -10.0%
  After : 0.29095560000000004

--- IMPLIED VOLATILITY CHANGES (LONG PUT) ---
  Before: 

{15.0: {'vol_collapse': {'S_0': np.float64(543.37),
   'S_t': np.float64(624.88),
   'put_long_iv_0': np.float64(0.3903),
   'Change_in_put_long_iv (%)': np.float64(-32.0),
   'put_long_iv_t': np.float64(0.2654),
   'put_short_iv_0': np.float64(0.3233),
   'Change_in_put_short_iv (%)': np.float64(-25.0),
   'put_short_iv_t': np.float64(0.2425),
   'call_iv_0': np.float64(0.2109),
   'Change_in_call_iv (%)': np.float64(-9.0),
   'call_iv_t': np.float64(0.192),
   'V_p_long_0': np.float64(4.425),
   'Change_in_put_long_price (%)': np.float64(-100.0),
   'V_p_long_t': np.float64(0.0),
   'V_p_short_0': np.float64(8.475),
   'Change_in_put_short_price (%)': np.float64(-100.0),
   'V_p_short_t': np.float64(0.0),
   'V_c_0': np.float64(4.225),
   'Change_in_call_price (%)': np.float64(1182.8066),
   'V_c_t': np.float64(54.1986),
   'pnl_dollars': np.float64(5192469.7),
   'pnl_percentage': np.float64(1282.09)},
  'normal_decline': {'S_0': np.float64(543.37),
   'S_t': np.float64(624.88),
   

In [12]:
df_scenario_analysis_result = simulation_result_to_df(scenario_simulation_result)

df_scenario_analysis_result['avg_pnl_for_spot_move'] = (
    df_scenario_analysis_result
    .groupby('Spot Move(%)')['pnl_dollars']
    .transform('mean').map('{:,.2f}'.format)
)

df_scenario_analysis_result.to_csv('../data/Result/3_Scenario_Analysis_Result.csv')
df_scenario_analysis_result

S_0     S_t  put_long_iv_0  \
Spot Move(%) scenario                                        
 15.0        vol_collapse    543.37  624.88         0.3903   
             normal_decline  543.37  624.88         0.3903   
             mixed_regime    543.37  624.88         0.3903   
             persistent_vol  543.37  624.88         0.3903   
 10.0        vol_collapse    543.37  597.71         0.3903   
             normal_decline  543.37  597.71         0.3903   
             mixed_regime    543.37  597.71         0.3903   
             persistent_vol  543.37  597.71         0.3903   
 5.0         vol_collapse    543.37  570.54         0.3903   
             normal_decline  543.37  570.54         0.3903   
             mixed_regime    543.37  570.54         0.3903   
             persistent_vol  543.37  570.54         0.3903   
 2.5         vol_collapse    543.37  556.95         0.3903   
             normal_decline  543.37  556.95         0.3903   
             mixed_regime    543.37  556.95         0.3903   
             persistent_vol  543.37  556.95         0.3903   
 0.0         vol_collapse    543.37  543.37         0.3903   
             normal_decline  543.37  543.37         0.3903   
             mixed_regime    543.37  543.37         0.3903   
             persistent_vol  543.37  543.37         0.3903   
-2.5         vol_collapse    543.37  529.79         0.3903   
             normal_decline  543.37  529.79         0.3903   
             mixed_regime    543.37  529.79         0.3903   
             persistent_vol  543.37  529.79         0.3903   
-5.0         vol_collapse    543.37  516.20         0.3903   
             normal_decline  543.37  516.20         0.3903   
             mixed_regime    543.37  516.20         0.3903   
             persistent_vol  543.37  516.20         0.3903   
-10.0        vol_collapse    543.37  489.03         0.3903   
             normal_decline  543.37  489.03         0.3903   
             mixed_regime    543.37  489.03         0.3903   
             persistent_vol  543.37  489.03         0.3903   
-15.0        vol_collapse    543.37  461.86         0.3903   
             normal_decline  543.37  461.86         0.3903   
             mixed_regime    543.37  461.86         0.3903   
             persistent_vol  543.37  461.86         0.3903   

                             Change_in_put_long_iv (%)  put_long_iv_t  \
Spot Move(%) scenario                                                   
 15.0        vol_collapse                        -32.0         0.2654   
             normal_decline                      -22.0         0.3044   
             mixed_regime                        -13.0         0.3395   
             persistent_vol                       -6.0         0.3669   
 10.0        vol_collapse                        -26.0         0.2888   
             normal_decline                      -18.0         0.3200   
             mixed_regime                        -10.0         0.3512   
             persistent_vol                       -4.0         0.3747   
 5.0         vol_collapse                        -18.0         0.3200   
             normal_decline                      -10.0         0.3512   
             mixed_regime                         -2.0         0.3825   
             persistent_vol                        5.0         0.4098   
 2.5         vol_collapse                         -7.0         0.3630   
             normal_decline                       -3.0         0.3786   
             mixed_regime                          6.0         0.4137   
             persistent_vol                       10.0         0.4293   
 0.0         vol_collapse                         -3.0         0.3786   
             normal_decline                        0.0         0.3903   
             mixed_regime                          7.0         0.4176   
             persistent_vol                       13.0         0.4410   
-2.5         vol_collapse                          4.0         0.4059

### Conclusion: Is the selling of 90/95 percent bullish put spread to finance the OTM Call a better structure on April 9?

* Both structures involve selling 1,000 contracts of either a put or a put spread to finance the purchase of a 5 percent out-of-the-money call, but they differ materially in how they distribute upside potential and downside risk.
Using the same scenario analysis framework as in Trade Structure 1, and adding an input to capture changes in implied volatility for the 90 percent out-of-the-money long put, we observe a clear trade-off between reducing downside risk and limiting upside potential.

* As expected, the put-spread-financed call produces lower upside exposure. In an extreme 15 percent rally in SPY, the maximum profit for the put spread structure is about 5.19 million dollars, compared with roughly 10.87 million dollars in the original bullish risk reversal. This reduction in upside is a natural outcome of allocating part of the collected premium to purchase a deeper OTM put.

* The benefit appears clearly on the downside. In a 15 percent decline, the put spread structure loses approximately 2.62 million dollars, whereas the naked put structure in Part 1 loses about 5.43 million dollars. This highlights the value of removing the unbounded downside exposure, which is especially important in a market environment where the rally on April 9 may represent only temporary relief rather than a durable shift in sentiment.

* Even in modest pullbacks, the put spread performs better. With a 2.5 percent decline in SPY, the put spread shows an average loss of about 354,000 dollars, compared with 442,000 dollars for the original structure. This improvement arises because owning an additional OTM put helps offset volatility expansion and skew steepening, both of which are common after large one day rallies when market uncertainty remains elevated. In addition, one week before maturity, Trade Structure 1 only generates a higher profit when the spot price is above roughly -2.5 to 0 % relative to the starting level, meaning the put spread remains superior across a wider range of mild pullback scenarios.

* In summary, the put spread structure is more suitable if one believes the April 9 rally is a short lived rebound rather than the beginning of a sustained uptrend. It offers significantly improved protection against renewed declines, which is valuable given the unresolved trade tensions, persistent macro uncertainty, and the tendency for relief rallies in stressed markets to experience sharp reversals. If one instead expects strong and persistent follow-through on the upside, the original risk reversal retains an advantage because of its larger exposure to positive convexity.
---

### Conclusion: Is buying SPY alone better?

* Buying SPY outright becomes the better choice when the post April 9 environment favors a slow gradual and orderly recovery or even a pull back in the SPY rather than another sharp upside extension. Both option structures, the Bullish Risk Reversal and the Put Spread plus OTM Call, rely on selling downside options to finance upside exposure and therefore perform best when SPY rallies quickly. If the market instead grinds higher as volatility normalizes, outright SPY is superior because it avoids theta decay and exposure to implied volatility movements, allowing the investor to capture the full price appreciation without relying on a fast leveraged directional move.

* SPY is also preferable when downside skew and macro uncertainty remain elevated. Both option structures require shorting volatility, which can lead to losses if volatility stays high or if market conditions become choppy. Owning SPY avoids these risks and provides a cleaner directional exposure when the outlook is stable but uncertain.

* However, outright SPY requires upfront capital, while both option structures are largely self-financing through the premium collected from selling puts. For traders with strong conviction that SPY will experience sustained upside over the next three weeks, this capital efficiency and leveraged payoff can make the Bullish Risk Reversal or the Put Spread plus OTM Call more attractive than buying SPY directly.

